RAIN - Real & Artificial Intelligence for Neuroscience

## Video handling

Welcome to my video handling notebook!

Here you'll find are a compilation of many functions I've written when working with videos.

Since we record behavioral experiments, sometimes it is useful to cut, reshape, draw on, and add text to videos, among other things.

In [1]:
import os
import numpy as np
import sys
import cv2
from tkinter import Tk, filedialog, Label, Entry, Button, Toplevel, Spinbox, messagebox
import random

In [3]:
def select_video_files() -> list:
    """Select video files using a file dialog.

    Returns:
        list: List of selected video files.
    """
    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()
    
    # Open file dialog to select video files
    video_files = filedialog.askopenfilenames(
        title="Select Video Files",
        filetypes=[("Video Files", "*.mp4 *.avi *.mkv *.mov")]
    )
    if not video_files:
        raise ValueError("No video files selected.")
    
    print(f"Selected {len(video_files)} videos.")

    return video_files

---
## 1. Trimming and Resizing
The most common video editing tasks are:
- Trim the video if it is too long
- Resize the video if it is too big (heavy)

In [3]:
def trim_video(file_path, width, height, fps, start_time=0, end_time=None):
    """
    Changes the resolution of a video and trims it.
    
    Parameters:
    - file_path: Path to the input video file.
    - width: Target width of the video.
    - height: Target height of the video.
    - start_time: Start time in seconds for the output video.
    - end_time: End time in seconds for the output video (None for full length).
    """

    folder_path = os.path.dirname(file_path)
    output_folder = os.path.join(folder_path, "trimmed")
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(file_path))

    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return
        
    # Get the original video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    # Convert time to frame numbers
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps) if end_time else total_frames
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = start_frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count >= end_frame:
            break
        
        # Resize the frame
        resized_frame = cv2.resize(frame, (width, height))

        # Write the frame to the output video
        out.write(resized_frame)
        frame_count += 1

    cap.release()
    out.release()
    print(f"Processing: {os.path.basename(file_path)}, Start: {start_time}s, End: {end_time if end_time else 'end'}s, {width}x{height}, {fps} FPS")

def get_video_info(file_path):
    """Gets the resolution and FPS of a video file."""
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return None, None, None
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    cap.release()
    return width, height, fps

class VideoTrimmer:
    def __init__(self, root, video_files):
        self.video_files = video_files
        self.current_file = video_files[0] if video_files else None
        
        # Get video resolution and FPS
        self.original_width, self.original_height, self.original_fps = get_video_info(self.current_file)
        if self.original_width is None:
            return
        
        self.aspect_ratio = self.original_height / self.original_width
        
        # Create a new window
        self.window = Toplevel(root)
        self.window.title("Video Editor")
        self.window.geometry("200x300")

        # Labels & Input Fields
        Label(self.window, text="Start Time (mm:ss)").pack()
        self.start_time = Entry(self.window)
        self.start_time.insert(0, "00:00")
        self.start_time.pack()

        Label(self.window, text="End Time (mm:ss)").pack()
        self.end_time = Entry(self.window)
        self.end_time.insert(0, "00:05")
        self.end_time.pack()

        Label(self.window, text=f"Resolution (default {self.original_width}x{self.original_height})").pack()

        self.width_spin = Spinbox(self.window, from_=100, to=4000, command=self.update_height)
        self.width_spin.pack()
        self.width_spin.delete(0, "end")
        self.width_spin.insert(0, str(self.original_width))

        self.height_spin = Spinbox(self.window, from_=100, to=4000, command=self.update_width)
        self.height_spin.pack()
        self.height_spin.delete(0, "end")
        self.height_spin.insert(0, str(self.original_height))

        Label(self.window, text=f"FPS (default {self.original_fps})").pack()
        self.fps_spin = Spinbox(self.window, from_=1, to=200)
        self.fps_spin.pack()
        self.fps_spin.delete(0, "end")
        self.fps_spin.insert(0, str(self.original_fps))

        Button(self.window, text="Run", command=self.run).pack()

    def update_height(self):
        """Automatically adjust height based on width to maintain aspect ratio."""
        try:
            width = int(self.width_spin.get())
            height = int(width * self.aspect_ratio)
            self.height_spin.delete(0, "end")
            self.height_spin.insert(0, str(height))
        except ValueError:
            pass  # Ignore invalid input

    def update_width(self):
        """Automatically adjust width based on height to maintain aspect ratio."""
        try:
            height = int(self.height_spin.get())
            width = int(height / self.aspect_ratio)
            self.width_spin.delete(0, "end")
            self.width_spin.insert(0, str(width))
        except ValueError:
            pass  # Ignore invalid input

    def run(self):
        start_time = self.convert_time(self.start_time.get())
        end_time = self.convert_time(self.end_time.get())
        width = int(self.width_spin.get())  
        height = int(self.height_spin.get())  
        fps = int(self.fps_spin.get())  

        for file in self.video_files:
            trim_video(file, width, height, fps, start_time, end_time)
        
        print("Processing complete.")
        self.window.destroy()
        self.window.quit()  # Ensures mainloop exits completely

    def convert_time(self, time_str):
        """Convert mm:ss format to seconds."""
        minutes, seconds = map(int, time_str.split(":"))
        return minutes * 60 + seconds

if __name__ == '__main__':
    try:
        root = Tk()
        root.withdraw()  # Hide the main window
        
        video_files = select_video_files()
        
        if not video_files:  # If the user cancels, exit without error
            print("Operation canceled.")
            root.destroy()
            sys.exit(0)  
        
        VideoTrimmer(root, video_files)
        root.mainloop()

    except KeyboardInterrupt:
        print("\nOperation interrupted by user.")
        root.destroy()
        sys.exit(0)  # Exit without error

Selected 2 videos.
Processing: WIN_20240405_16_21_01_Pro_aligned.mp4, Start: 0s, End: 5s, 500x500, 30 FPS
Processing: WIN_20240405_16_27_23_Pro_aligned.mp4, Start: 0s, End: 5s, 500x500, 30 FPS
Processing complete.


---
## 2. Aligning
When we record videos with different cameras, or our camera moves a bit between recordings, we can use the following function to align the videos.

In [ ]:
def merge_frames_within_video(video_file, num_frames: int = 5) -> np.ndarray:
    """
    Merge a specified number of random frames from each video file into a single image.

    Args:
        num_frames (int): Number of random frames to merge from each video. Default is 5.
    
    Returns:
        np.ndarray: Merged image.
    """
    merged_image = None
    
    cap = cv2.VideoCapture(video_file)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    selected_frame_indices = random.sample(range(total_frames), num_frames)

    for frame_idx in selected_frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        success, frame = cap.read()

        if not success:
            print(f"Could not read frame {frame_idx} from {video_file}")
            continue

        # Calculate transparency
        transparency = round(1 / num_frames, 4)
        transparent_frame = (frame * transparency).astype(np.uint8)
        
        if merged_image is None:
            # Initialize merged image
            merged_image = np.zeros_like(transparent_frame)
        
        # Add transparent frame to the merged image
        merged_image = cv2.add(merged_image, transparent_frame)
    
    cap.release()
    
    return merged_image

def align_videos():

    print("Instructions:")
    print("1. Left-click to select points.")
    print("2. Enter to confirm the current point.")
    print("3. Select two points on each video to align them.")
    print("Press 'q' to quit without aligning.")

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()
    
    # Open file dialog to select video files
    video_files = select_video_files()

    # Initialize variables
    zoom_scale = 5  # How much to zoom in
    zoom_window_size = 25  # Half the width/height of the zoomed-in area
    point_pairs = []  # To store pairs of points for each video
    first_frames = []

    # Define callback function for point selection
    def select_points(event, x, y, flags, param):
        nonlocal frame, temp_frame, current_point, confirmed_points, zoom_scale, zoom_window_size

        #if event == cv2.EVENT_MOUSEMOVE:   

        if event == cv2.EVENT_LBUTTONDOWN:
            # Update the current point with the clicked position
            current_point = (x, y)
            # Draw the current point
            cv2.circle(temp_frame, current_point, 3, (0, 255, 0), -1)
            # Draw the confirmed points on the frame
            for point in confirmed_points: 
                cv2.circle(temp_frame, point, 3, (0, 0, 255), -1)
            # Display the frame
            cv2.imshow('Select Points', temp_frame)
        
        # Create zoomed-in display
        x1 = max(0, x - zoom_window_size)
        x2 = min(temp_frame.shape[1], x + zoom_window_size)
        y1 = max(0, y - zoom_window_size)
        y2 = min(temp_frame.shape[0], y + zoom_window_size)

        zoomed_area = temp_frame[y1:y2, x1:x2]
        
        # Resize zoomed-in area
        zoomed_area_resized = cv2.resize(zoomed_area, None, fx=zoom_scale, fy=zoom_scale, interpolation=cv2.INTER_LINEAR)

        # Add crosshair to the center
        center_x = zoomed_area_resized.shape[1] // 2
        center_y = zoomed_area_resized.shape[0] // 2
        color = (0, 255, 0)  # Black crosshair
        thickness = 2
        line_length = 20  # Length of crosshair lines

        # Draw vertical line
        cv2.line(zoomed_area_resized, (center_x, center_y - line_length), (center_x, center_y + line_length), color, thickness)
        # Draw horizontal line
        cv2.line(zoomed_area_resized, (center_x - line_length, center_y), (center_x + line_length, center_y), color, thickness)

        if x2 > (temp_frame.shape[1] - zoomed_area_resized.shape[1] - 10) and y1 < (10 + zoomed_area_resized.shape[0]):
            # Overlay zoomed-in area in the top-left corner of the frame
            overlay_x1 = 10
            overlay_x2 = 10 + zoomed_area_resized.shape[1]
            overlay_y1 = 10
            overlay_y2 = 10 + zoomed_area_resized.shape[0]
        
        else:
            # Overlay zoomed-in area in the top-right corner of the frame
            overlay_x1 = temp_frame.shape[1] - zoomed_area_resized.shape[1] - 10
            overlay_x2 = temp_frame.shape[1] - 10
            overlay_y1 = 10
            overlay_y2 = 10 + zoomed_area_resized.shape[0]
        
        # Reset the frame
        temp_frame = frame.copy()

        # Draw the current point
        if current_point is not None:
            cv2.circle(temp_frame, current_point, 3, (0, 255, 0), -1)
        # Draw the confirmed points on the frame
        for point in confirmed_points:
            cv2.circle(temp_frame, point, 3, (0, 0, 255), -1)
        # Display the zoomed-in area
        temp_frame[overlay_y1:overlay_y2, overlay_x1:overlay_x2] = zoomed_area_resized
        # Display the frame
        cv2.imshow('Select Points', temp_frame)

    def confirm_point():
        """Confirm the current point and add it to the list."""
        nonlocal temp_frame, confirmed_points, current_point
        if current_point is not None:
            confirmed_points.append(current_point)
            # Draw the confirmed points on the frame
            for point in confirmed_points: 
                cv2.circle(temp_frame, point, 3, (0, 0, 255), -1)
            # Display the frame
            cv2.imshow('Select Points', temp_frame)
            current_point = None
            print(f"Point confirmed: {confirmed_points[-1]}")  # Feedback to the user
    
    # Step 1: Extract first frames and collect two points for each video
    for video_path in video_files:
        cap = cv2.VideoCapture(video_path)
        frame = merge_frames_within_video(video_path)
        first_frames.append((frame, video_path))
        confirmed_points = []  # Store the two confirmed points for this video
        current_point = None  # Temporary point being adjusted
        temp_frame = frame.copy()  # Create a copy of the frame

        # Run the mouse callback with the frame and confirmed points
        cv2.imshow('Select Points', frame)
        cv2.setMouseCallback('Select Points', select_points)

        # Wait for user to confirm two points
        while len(confirmed_points) < 2:
            key = cv2.waitKey(1) & 0xFF
            if key == 13:  # Enter key to confirm the current point
                confirm_point()
            elif key == ord('q'):  # Press 'q' to quit
                response = messagebox.askquestion("Exit", "Do you want to exit aligner?")
                if response == 'yes':
                    print("Exiting point selection.")
                    cv2.destroyAllWindows()
                    return
            
        # Save the confirmed points
        point_pairs.append(confirmed_points)
        cap.release()

    cv2.destroyAllWindows()
    
    # Step 2: Calculate mean points
    if not point_pairs:
        print("No points were selected.")
        return
    
    mean_points = np.mean(point_pairs, axis=0)
    mean_point1, mean_point2 = mean_points.astype(int)

    response = messagebox.askquestion("Alignment", "Do you want the points to stand on the same horizontal line?")  
    if response == 'yes':
        # Calculate the mean y-value
        y_mean = (mean_point1[1] + mean_point2[1]) // 2  # Use integer division if you want the result as int

        # Update the y-values of both points
        mean_point1[1] = y_mean
        mean_point2[1] = y_mean
    
    print(f"Mean points: {mean_point1}, {mean_point2}")
    
    # Step 3: Align videos (rotate, resize, then translate)
    output_folder = os.path.join(os.path.dirname(video_files[0]), 'aligned')
    os.makedirs(output_folder, exist_ok=True)
    mean_vector = mean_point2 - mean_point1
    mean_length = np.linalg.norm(mean_vector)
    mean_angle = np.arctan2(mean_vector[1], mean_vector[0])
    
    for (frame, video_path), points in zip(first_frames, point_pairs):
        point1, point2 = points
        vector = np.array(point2) - np.array(point1)
        angle = np.arctan2(vector[1], vector[0])
        length = np.linalg.norm(vector)
        
        scale = mean_length / length
        rotation_angle = mean_angle + angle

        # Step 3.1: Rotate and resize
        height, width = frame.shape[:2]
        center = (width // 2, height // 2)
        M_rotate_scale = cv2.getRotationMatrix2D(center, np.degrees(rotation_angle), scale)
        rotated_resized_frame = cv2.warpAffine(frame, M_rotate_scale, (width, height))
        
        # Step 3.2: Translate
        new_point1 = np.dot(M_rotate_scale[:, :2], np.array(point1).T) + M_rotate_scale[:, 2]
        dx, dy = mean_point1[0] - new_point1[0], mean_point1[1] - new_point1[1]
        M_translate = np.float32([[1, 0, dx], [0, 1, dy]])
        aligned_frame = cv2.warpAffine(rotated_resized_frame, M_translate, frame.shape[1::-1])
        
        # Save aligned video
        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_name = os.path.basename(video_path)
        output_path = os.path.join(output_folder, video_name)
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Apply the same transformations to each frame
            rotated_resized_frame = cv2.warpAffine(frame, M_rotate_scale, frame.shape[1::-1])
            aligned_frame = cv2.warpAffine(rotated_resized_frame, M_translate, frame.shape[1::-1])
            out.write(aligned_frame)
        
        cap.release()
        out.release()

        print(f"Aligned '{video_name}' with scale {scale:.2f}, rotation {rotation_angle:.2f}, and translation {dx:.2f}, {dy:.2f}.")
    
    print(f"Aligned videos saved in '{output_folder}'.")

In [5]:
align_videos()

Instructions:
1. Left-click to select points.
2. Enter to confirm the current point.
3. Select two points on each video to align them.
Press 'q' to quit without aligning.
Selected 2 videos.
Point confirmed: (44, 250)
Point confirmed: (454, 250)
Point confirmed: (45, 249)
Point confirmed: (453, 248)
Mean points: [ 44 249], [453 249]
Aligned 'WIN_20240405_16_21_01_Pro_aligned.mp4' with scale 1.00, rotation 0.00, and translation -0.50, -1.00.
Aligned 'WIN_20240405_16_27_23_Pro_aligned.mp4' with scale 1.00, rotation -0.00, and translation -0.50, 0.51.
Aligned videos saved in 'C:/Users/dhers/OneDrive/Doctorado/Experimentos/3xTg_B2/2024_04-E+M/4_cropped/trimmed\Aligned'.


---
## 3. Cropping
The following function will let you crop the video to the desired size.
- It's useful for when you have unused space in the video, or if you record multiple animals simultaneously with one camera.

In [4]:
def merge_frames_across_videos(video_files: list) -> np.ndarray:
    """
    Merge the first frame of each video file into a single image.

    Args:
        video_files (list): List of video files.
    
    Returns:
        np.ndarray: Merged image.
    """
    merged_image = None
    
    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        success, frame = cap.read()
        cap.release()
        
        if not success:
            print(f"Could not read first frame of {video_file}")
            continue
        
        # Calculate transparency
        transparency = round(1 / len(video_files), 4)
        transparent_frame = (frame * transparency).astype(np.uint8)
        
        if merged_image is None:
            # Initialize merged image
            merged_image = np.zeros_like(transparent_frame)
        
        # Add transparent frame to the merged image
        merged_image = cv2.add(merged_image, transparent_frame)
    
    return merged_image

def define_rectangle(x1, y1, x2, y2):
    """Define a rectangle based on two points.
    """
    width = int(round(abs(x2 - x1)))
    height = int(round(abs(y2 - y1)))
    center = [int(round((x1+x2)//2)), int(round((y1+y2)//2))] # Round to integer
    return center, width, height

def draw_rectangle(image, center, width, height, angle = 0, color = (0, 255, 0), thickness = 2):
    """Draw a rectangle on an image.
    """
    box = cv2.boxPoints(((center[0], center[1]), (width, height), angle))
    box = np.intp(box)  # Convert to integer
    cv2.drawContours(image, [box], 0, color, thickness)
    cv2.circle(image, (int(round(center[0])), int(round(center[1]))), radius=2, color=color, thickness=-1)

def crop_video(file_path, center, width, height, angle = 0):
    """Crop a video to a specific size and center.

    Args:
        file_path (str): Path to the video file.
        center (tuple): Center of the cropped region.
        width (int): Width of the cropped region.
        height (int): Height of the cropped region.
        angle (int, optional): Angle of rotation. Defaults to 0.
    """
    
    folder_path = os.path.dirname(file_path)
    output_folder = os.path.join(folder_path, "cropped")
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(file_path))
    
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {file_path}.")
        return
        
    # Output video writer with the cropped size
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if angle != 0:
            # Rotate the entire frame
            rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)
            frame = cv2.warpAffine(frame, rotation_matrix, (frame.shape[1], frame.shape[0]))
        
        # Crop the rotated region
        frame = frame[
            center[1] - height // 2:center[1] + height // 2,
            center[0] - width // 2:center[0] + width // 2,
        ]
        out.write(frame)

    cap.release()
    out.release()
    print(f"Cropped {os.path.basename(output_path)} with size {width}x{height}.")

def cropper():
    """
    Crop multiple videos to a specific size and center.
    """

    # Get video files
    video_files = select_video_files()

    # Merge frames
    image = merge_frames_across_videos(video_files)
    
    # Get original dimensions and print them
    width = image.shape[1]
    height = image.shape[0]
    print(f"Original Size: {width}x{height}")

    # Initialize variables
    clone = image.copy()
    corners = []  # Current ROI 
    dragging = [False]  # For moving ROI
    drag_start = None
    angle = 0  # Current angle for rotation
    rotate_factor = 1  # Amount of change per scroll
    resize_factor = 2  # Amount of change per scroll
    scale_factor = 1.0  # Scaling factor
    square = False  # For enforcing a square shape

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal clone, corners, dragging,drag_start, angle, rotate_factor, resize_factor, square

        # Adjust mouse coordinates according to scale factor
        x = int(x / scale_factor)
        y = int(y / scale_factor)

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            # angle = 0 # Reset angle
            if not dragging[0]:
                dragging[0] = True
                corners = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 1:
            x1, y1 = corners[0]
            x2, y2 = x, y # While dragging, update the end point

            # If Ctrl is held, enforce a square shape
            if flags & cv2.EVENT_FLAG_CTRLKEY:
                side = max(abs(x2 - x1), abs(y2 - y1))
                x2 = x1 + side if x2 > x1 else x1 - side
                y2 = y1 + side if y2 > y1 else y1 - side
                square = True
            else:
                square = False

            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                x1, y1 = corners[0]  # Start point
                x2, y2 = x, y  # End point
                if square:
                    side = max(abs(x2 - x1), abs(y2 - y1))
                    x2 = x1 + side if x2 > x1 else x1 - side
                    y2 = y1 + side if y2 > y1 else y1 - side
                corners.append((x2, y2))
                
        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(corners) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(corners) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            x1, y1, x2, y2 = (corners[0][0] + dx, corners[0][1] + dy, corners[1][0] + dx, corners[1][1] + dy)
            corners[0] = x1, y1
            corners[1] = x2, y2
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(corners) == 2:
            dragging[0] = False

        # Resize or rotate the ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(corners) == 2:
            x1, y1 = corners[0]
            x2, y2 = corners[1]
            if flags & cv2.EVENT_FLAG_CTRLKEY:  # Rotate with Ctrl key pressed
                if flags > 0:  # Scroll up
                    angle -= rotate_factor
                else:  # Scroll down
                    angle += rotate_factor
            else:  # Resize without modifier key
                width = max(abs(x2 - x1), 1)
                height = max(abs(y2 - y1), 1)
                ratio = width/height
                if flags > 0:  # Scroll up
                    x1 -= resize_factor*ratio
                    y1 -= resize_factor
                    x2 += resize_factor*ratio
                    y2 += resize_factor
                else:  # Scroll down
                    x1 += resize_factor*ratio
                    y1 += resize_factor
                    x2 -= resize_factor*ratio
                    y2 -= resize_factor
                corners = [(x1, y1), (x2, y2)]
            center, width, height = define_rectangle(x1, y1, x2, y2)
            clone = image.copy()
            draw_rectangle(clone, center, width, height, angle, (0, 255, 255), 2)

        # Draw the updated ROI and display width, height, and angle
        if len(corners) > 0:
            x1, y1 = corners[0]
            if len(corners) > 1:
                x2, y2 = corners[1]
            else:
                x2, y2 = x, y
                if square:
                    side = max(abs(x2 - x1), abs(y2 - y1))
                    x2 = x1 + side if x2 > x1 else x1 - side
                    y2 = y1 + side if y2 > y1 else y1 - side
            
            # Display height, width, and angle at the bottom of the frame
            center, width, height = define_rectangle(x1, y1, x2, y2)
            text = f"M: [{x}, {y}], C: {center}, W: {width}, H: {height}, A: {angle}"  # Convert to int for display
        else:
            text = f"M: [{x}, {y}]"

        font_scale, font_thickness = 0.5, 1
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        text_x, text_y = 10, clone.shape[0] - 10
        cv2.rectangle(clone, (text_x - 5, text_y - text_size[1] - 5), 
                    (text_x + text_size[0] + 8, text_y + 5), (0, 0, 0), -1)  # Background for text
        cv2.putText(clone, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (255, 255, 255), font_thickness)

    # Set up the OpenCV window and bind the mouse callback
    cv2.namedWindow("Select Region")
    cv2.setMouseCallback("Select Region", handle_mouse)

    while True:
        display_image = cv2.resize(clone, None, fx=scale_factor, fy=scale_factor)
        cv2.imshow("Select Region", display_image)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('+') and scale_factor < 2:
            scale_factor += 0.1  # Zoom in
        elif key == ord('-') and scale_factor > 0.5:
            scale_factor -= 0.1  # Zoom out
        elif key == ord('r'):
            scale_factor = 1.0  # Reset zoom
        

        elif key == ord('c') and len(corners) == 2:  # Save the ROI
            response = messagebox.askquestion("Crop", "Do you want to crop this region?")
            if response == 'yes':
                break

        elif key == ord('q'):  # Quit and save
            response = messagebox.askquestion("Exit", "Do you want to exit the cropper?")
            if response == 'yes':
                print("Cropping canceled.")
                cv2.destroyAllWindows()
                return                

    cv2.destroyAllWindows()

    # Ensure valid ROI
    x1, y1 = corners[0]
    x2, y2 = corners[1]
    center, width, height = define_rectangle(x1, y1, x2, y2)

    # Apply cropping
    for file in video_files:
        crop_video(file, center, width, height, angle)

    print(f"Videos cropped and saved")

In [5]:
cropper()

Selected 36 videos.
Original Size: 1080x960
Cropped 2024_04-R01_C1i-OF.mp4 with size 960x800.
Cropped 2024_04-R01_C1i-Splash.mp4 with size 960x800.
Cropped 2024_04-R02_C1d-OF.mp4 with size 960x800.
Cropped 2024_04-R02_C1d-Splash.mp4 with size 960x800.
Cropped 2024_04-R03_C1a-OF.mp4 with size 960x800.
Cropped 2024_04-R03_C1a-Splash.mp4 with size 960x800.
Cropped 2024_04-R04_C2i-OF.mp4 with size 960x800.
Cropped 2024_04-R04_C2i-Splash.mp4 with size 960x800.
Cropped 2024_04-R05_C2d-OF.mp4 with size 960x800.
Cropped 2024_04-R05_C2d-Splash.mp4 with size 960x800.
Cropped 2024_04-R06_C2a-OF.mp4 with size 960x800.
Cropped 2024_04-R06_C2a-Splash.mp4 with size 960x800.
Cropped 2024_04-R07_C3i-OF.mp4 with size 960x800.
Cropped 2024_04-R07_C3i-Splash.mp4 with size 960x800.
Cropped 2024_04-R08_C3d-OF.mp4 with size 960x800.
Cropped 2024_04-R08_C3d-Splash.mp4 with size 960x800.
Cropped 2024_04-R09_C3n-OF.mp4 with size 960x800.
Cropped 2024_04-R09_C3n-Splash.mp4 with size 960x800.
Cropped 2024_04-R10_